In [126]:
import pandas as pd
import glob
import os

# Path to your shared drive folder
folder = "/Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1"
# Columns you care about
import pandas as pd
import glob
import os

# Path to your shared drive folder
folder = "/Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult"

# Map messy/variant names → canonical
rename_map = {
    "blood_culture_taken_time_exact": "BloodCultureOrderDateTime",
    "BloodCultureOrderDateTime": "BloodCultureOrderDateTime",
    "notedatetime_exact": "NoteDateTime",
    "NoteDateTime": "NoteDateTime",
    "Reviewer ": "Reviewer",
    "Tie Breaker Reviewer": "Reviewer",  # optional consolidation
}

# Final columns you want
keep_cols = ["MRN", "BloodCultureOrderDateTime", "NoteDateTime", "NoteType", "Reviewer", "Risk of Bactermia", "source_file"]

all_data = []
xlsx_files = glob.glob(os.path.join(folder, "**", "*.xlsx"), recursive=True)

for file in xlsx_files:
    try:
        df = pd.read_excel(file, engine="openpyxl")
        df["source_file"] = os.path.basename(file)

        # Rename columns (normalize known variants)
        df = df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

        # Keep only desired columns (if missing, fill with NA)
        slim = pd.DataFrame()
        for col in keep_cols:
            if col in df.columns:
                slim[col] = df[col]
            else:
                slim[col] = pd.NA

        all_data.append(slim)
        print(f"✅ Processed {file} with {slim.shape[0]} rows")
    except Exception as e:
        print(f"⚠️ Could not read {file}: {e}")

# Combine
if all_data:
    combined = pd.concat(all_data, ignore_index=True)
    print("\nFinal shape:", combined.shape)
    print(combined.head())
else:
    print("No data loaded.")




✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Tiebreaker.xlsx with 52 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Kameron.xlsx with 121 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Unknown.xlsx with 40 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Aydin.xlsx with 120 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodC

In [127]:
combined["MRN"].nunique()

172

In [128]:
import pandas as pd
import os

# assume `combined` is your DataFrame
tiebreaker_file = "BloodCultureAnalysis_Phase1_Tiebreaker.xlsx"

# 1. collect MRNs from Tiebreaker
tiebreaker_mrns = set(
    combined.loc[combined["source_file"] == tiebreaker_file, "MRN"].dropna().unique()
)
print(f"Found {len(tiebreaker_mrns)} MRNs in Tiebreaker")

# 2. keep:
#   - all rows from Tiebreaker
#   - all rows from other files, but only if MRN not in tiebreaker_mrns
filtered = combined[
    (combined["source_file"] == tiebreaker_file) |
    (~combined["MRN"].isin(tiebreaker_mrns))
].copy()

print("Before:", combined.shape)
print("After:", filtered.shape)

# 3. optional: row counts by file
print("\nRow counts by source_file after filtering:")
print(filtered["source_file"].value_counts())

# 4. optional: save
out_path = os.path.join(
    folder, "combined_filtered_tiebreaker_priority.csv"
)
filtered.to_csv(out_path, index=False)
print(f"💾 Saved filtered dataset: {out_path}")


Found 26 MRNs in Tiebreaker
Before: (734, 7)
After: (626, 7)

Row counts by source_file after filtering:
source_file
BloodCultureAnalysis_Phase2_NickManoj.xlsx     401
BloodCultureAnalysis_Phase1_Kameron.xlsx        67
BloodCultureAnalysis_Phase1_Aydin.xlsx          66
BloodCultureAnalysis_Phase1_Tiebreaker.xlsx     52
BloodCultureAnalysis_Phase1_Unknown.xlsx        40
Name: count, dtype: int64
💾 Saved filtered dataset: /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/combined_filtered_tiebreaker_priority.csv


In [129]:
filtered

,MRN,BloodCultureOrderDateTime,NoteDateTime,NoteType,Reviewer,Risk of Bactermia,source_file
0,78178720.0,2023-02-19 07:18:00,2023-02-19 05:52:00,ED Provider,Marshall,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
1,78178720.0,2023-02-19 07:24:00,2023-02-19 05:52:00,ED Provider,Marshall,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
2,13562038.0,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Marshall,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
3,13562038.0,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Marshall,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
4,18418582.0,2023-01-13 12:55:00,2023-01-13 11:06:00,ED Provider,Marshall,Low Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
...,...,...,...,...,...,...,...
729,38110540.0,2022-12-08 01:50:00,2022-12-08 02:21:00,ED Provider,Manoj,NaN,BloodCultureAnalysis_Phase2_NickManoj.xlsx
730,17008376.0,2022-12-07 03:23:00,2022-12-07 02:57:00,ED Provider,Manoj,NaN,BloodCultureAnalysis_Phase2_NickManoj.xlsx
731,17008376.0,2022-12-07 02:53:00,2022-12-07 02:57:00,ED Provider,Manoj,NaN,BloodCultureAnalysis_Phase2_NickManoj.xlsx
732,29760402.0,2022-12-05 06:11:00,2022-12-05 04:47:00,ED Provider,Manoj,NaN,BloodCultureAnalysis_Phase2_NickManoj.xlsx


In [130]:
filtered_labelled = filtered[filtered["Risk of Bactermia"].notna()]

In [131]:
# get unique MRNs as a Python list
mrns = filtered_labelled["MRN"].dropna().unique().tolist()
print(f"{len(mrns)} unique MRNs")

122 unique MRNs


In [132]:
from google.cloud import bigquery

client = bigquery.Client(project="som-nero-phi-jonc101-secure")

# build query with IN @mrns
query = """
    SELECT *
    FROM `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-07-17`
    WHERE mrn IN UNNEST(@mrns)
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("mrns", "STRING", mrns)
    ]
)

mrns_df = client.query(query, job_config=job_config).to_dataframe()
print(mrns_df.head())


        mrn    anon_id  jitter  offset_pat_enc_csn_id  offset_order_id  \
0  16510281   JC902137     -31                1943675           592050   
1  20798146  JC1187757     -29                1943675           592050   
2  16140931   JC876685     -28                1943675           592050   
3  77329753  JC2910146     -22                1943675           592050   
4  15378490   JC823234     -21                1943675           592050   

   offset_inpatient_data_id  message_id_offset  
0                   5774830              47385  
1                   5774830              47385  
2                   5774830              47385  
3                   5774830              47385  
4                   5774830              47385  


In [ ]:
# from google.cloud import bigquery

# client = bigquery.Client(project="som-nero-phi-jonc101")

# # build query with IN @mrns
# query = """
#     SELECT * FROM `som-nero-phi-jonc101.Deid_Notes_JChen.Deid_Notes_SHC_JChen` where anon_id in UNNEST(@anon_ids)
#     and note_type_desc = "ED Provider Notes"
    

# """
# anon_ids = mrns_df["anon_id"].tolist()
# job_config = bigquery.QueryJobConfig(
#     query_parameters=[
#         bigquery.ArrayQueryParameter("anon_ids", "STRING", anon_ids)
#     ]
# )

# mrn_anon_id_mapping = client.query(query, job_config=job_config).to_dataframe()
# print(mrn_anon_id_mapping.head())


In [43]:
from google.cloud import bigquery

client = bigquery.Client(project="som-nero-phi-jonc101")

# build query with IN @mrns
query = """
    SELECT * FROM `som-nero-phi-jonc101.blood_culture_stewardship.EDProviderNotes` where anon_id in UNNEST(@anon_ids)
    

"""
anon_ids = mrns_df["anon_id"].tolist()
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("anon_ids", "STRING", anon_ids)
    ]
)

mrn_anon_id_mapping = client.query(query, job_config=job_config).to_dataframe()
mrn_anon_id_mapping.rename(columns={"notedatetime": "notedatetime_utc_plus_8"}, inplace=True)
print(mrn_anon_id_mapping.head())


     anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0  JC2642643          131344783990            835728861   
1   JC979941          131348395480            853375583   
2  JC2162185          131349687708            858336865   
3   JC737272          131351904872            866609862   
4  JC3218426          131347572748            848944865   

    notedatetime_utc_plus_8  \
0 2023-01-01 22:14:00+00:00   
1 2023-01-15 08:20:00+00:00   
2 2023-03-14 19:36:00+00:00   
3 2023-03-03 07:44:00+00:00   
4 2023-01-06 04:19:00+00:00   

                                      deid_note_text note_type  \
0                          Emergency Department  ...   ED Note   
1             Provider Note     Name: Zina Hubbel...   ED Note   
2             Provider Note     Name: Pamella Hin...   ED Note   
3             Provider Note     Name: Bonnie Nord...   ED Note   
4             Provider Note     Name: Mona L Lang...   ED Note   

      note_type_desc  
0  ED Provider Notes  
1  ED Provider 

In [44]:
mrn_anon_id_mapping[mrn_anon_id_mapping["anon_id"] == "JC2642643"]

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,notedatetime_utc_plus_8,deid_note_text,note_type,note_type_desc
0,JC2642643,131344783990,835728861,2023-01-01 22:14:00+00:00,Emergency Department ...,ED Note,ED Provider Notes
13,JC2642643,131344783990,835728859,2023-01-01 22:14:00+00:00,Emergency Department ...,ED Note,ED Provider Notes


In [ ]:
filtered_labelled[filtered_labelled["MRN"] == 73148181]["NoteDateTime"]+ pd.Timedelta(hours=26*24 +8)


163   2023-01-01 22:14:00
164   2023-01-01 22:14:00
323   2023-01-01 22:14:00
324   2023-01-01 22:14:00
Name: NoteDateTime, dtype: datetime64[ns]

In [47]:
from google.cloud import bigquery

client = bigquery.Client(project="som-nero-phi-jonc101")

# build query with IN @mrns
query = """
    SELECT * FROM `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where anon_id in UNNEST(@anon_ids)
    

"""
anon_ids = mrns_df["anon_id"].tolist()
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("anon_ids", "STRING", anon_ids)
    ]
)

culture_labels = client.query(query, job_config=job_config).to_dataframe()
print(culture_labels.head())


     anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0  JC1027919          131356579515            884150450   
1  JC1027919          131356579515            884150451   
2  JC2900296          131291655683            673348867   
3  JC2900296          131291655683            673348866   
4  JC2657740          131349920754            859269891   

  blood_culture_order_datetime  order_year       ed_arrival_datetime  \
0    2023-06-13 22:31:00+00:00        2023 2023-06-13 22:17:00+00:00   
1    2023-06-13 22:31:00+00:00        2023 2023-06-13 22:17:00+00:00   
2    2020-08-16 15:55:00+00:00        2020 2020-08-16 13:58:00+00:00   
3    2020-08-16 15:55:00+00:00        2020 2020-08-16 13:58:00+00:00   
4    2023-03-23 03:19:00+00:00        2023 2023-03-23 02:26:00+00:00   

   positive_blood_culture  positive_blood_culture_in_week  \
0                       0                               0   
1                       0                               0   
2                       0    

In [53]:
culture_labels["label"] = culture_labels["positive_blood_culture"]| culture_labels["positive_blood_culture_in_week"]

In [56]:
mrns_df

,mrn,anon_id,jitter,offset_pat_enc_csn_id,offset_order_id,offset_inpatient_data_id,message_id_offset
0,16510281,JC902137,-31,1943675,592050,5774830,47385
1,20798146,JC1187757,-29,1943675,592050,5774830,47385
2,16140931,JC876685,-28,1943675,592050,5774830,47385
3,77329753,JC2910146,-22,1943675,592050,5774830,47385
4,15378490,JC823234,-21,1943675,592050,5774830,47385
...,...,...,...,...,...,...,...
106,27284520,JC1538188,24,1943675,592050,5774830,47385
107,10711356,JC653030,25,1943675,592050,5774830,47385
108,10820280,JC655780,25,1943675,592050,5774830,47385
109,73148181,JC2642643,26,1943675,592050,5774830,47385


In [60]:
filtered_labelled.sort_values(by="MRN", ascending=False)

,MRN,BloodCultureOrderDateTime,NoteDateTime,NoteType,Risk of Bactermia,source_file
400,98259286.0,2023-01-04 10:00:00,2023-01-04 09:53:00,ED Provider,LowRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx
401,98259286.0,2023-01-04 10:00:00,2023-01-04 09:53:00,ED Provider,LowRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx
12,85386969.0,2023-03-18 14:10:00,2023-03-18 14:24:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
13,85386969.0,2023-03-18 14:51:00,2023-03-18 14:24:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx
409,85258846.0,2023-02-18 00:30:00,2023-02-17 21:59:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx
...,...,...,...,...,...,...
192,2302883.0,2022-12-27 19:03:00,2022-12-27 18:56:00,ED Provider,NotDetermined,BloodCultureAnalysis_Phase1_Unknown.xlsx
407,1071141.0,2023-01-14 20:55:00,2023-01-14 19:25:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx
406,1071141.0,2023-01-14 20:55:00,2023-01-14 19:25:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx
381,1039585.0,2022-12-21 20:12:00,2022-12-21 23:17:00,ED Provider,IntermediateRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx


In [78]:
# Ensure both are strings
filtered_labelled["MRN"] = filtered_labelled["MRN"].astype(str).str.strip().str.split(".").str[0]
mrns_df["mrn"] = mrns_df["mrn"].astype(str).str.strip()

# Now merge
human_merged = pd.merge(
    filtered_labelled,
    mrns_df[["mrn", "anon_id", "jitter"]],
    left_on="MRN",
    right_on="mrn",
    how="inner"
)

# Optional: drop duplicate 'mrn' column since you already have 'MRN'
human_merged = human_merged.drop(columns=["mrn"])


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_53264/2828576959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_labelled["MRN"] = filtered_labelled["MRN"].astype(str).str.strip().str.split(".").str[0]


In [87]:
human_merged["notedatetime_jittered_utc_plus_8"] = (
    human_merged["NoteDateTime"] +
    pd.to_timedelta(human_merged["jitter"], unit="D") +   # jitter in days
    pd.Timedelta(hours=8)                                 # fixed +8h
)

In [88]:
human_merged

,MRN,BloodCultureOrderDateTime,NoteDateTime,NoteType,Risk of Bactermia,source_file,anon_id,jitter,notedatetime_jittered_utc_plus_8
0,78178720,2023-02-19 07:18:00,2023-02-19 05:52:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC2965753,11,2023-03-02 13:52:00
1,78178720,2023-02-19 07:24:00,2023-02-19 05:52:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC2965753,11,2023-03-02 13:52:00
2,13562038,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC737272,-29,2023-03-03 07:44:00
3,13562038,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC737272,-29,2023-03-03 07:44:00
4,18418582,2023-01-13 12:55:00,2023-01-13 11:06:00,ED Provider,Low Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC1028634,-12,2023-01-01 19:06:00
...,...,...,...,...,...,...,...,...,...
283,36388841,2023-01-09 16:24:00,2023-01-09 15:05:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC2049825,27,2023-02-05 23:05:00
284,23828635,2023-01-31 15:25:00,2023-01-31 16:32:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC1356726,6,2023-02-07 00:32:00
285,23828635,2023-01-31 15:57:00,2023-01-31 16:32:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC1356726,6,2023-02-07 00:32:00
286,82062696,2023-02-26 20:58:00,2023-02-26 20:01:00,ED Provider,LowRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC3204260,-20,2023-02-07 04:01:00


In [96]:
human_merged["notedatetime_jittered_utc_plus_8"] = pd.to_datetime(
    human_merged["notedatetime_jittered_utc_plus_8"], utc=True
)

mrn_anon_id_mapping["notedatetime_utc_plus_8"] = pd.to_datetime(
    mrn_anon_id_mapping["notedatetime_utc_plus_8"], utc=True
)

human_merged_with_mrn_anon_id_mapping = pd.merge(
    human_merged,
    mrn_anon_id_mapping,
    left_on=["anon_id", "notedatetime_jittered_utc_plus_8"],
    right_on=["anon_id", "notedatetime_utc_plus_8"],
    how="inner"
).drop_duplicates(subset=["order_proc_id_coded"])


In [106]:
human_merged_with_mrn_anon_id_mapping_with_culture_labels = pd.merge(human_merged_with_mrn_anon_id_mapping, culture_labels[["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "label"]], \
    left_on=["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded"], right_on=["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded"], how="inner")

In [110]:
human_merged_with_mrn_anon_id_mapping_with_culture_labels

,MRN,BloodCultureOrderDateTime,NoteDateTime,NoteType,Risk of Bactermia,source_file,anon_id,jitter,notedatetime_jittered_utc_plus_8,pat_enc_csn_id_coded,order_proc_id_coded,notedatetime_utc_plus_8,deid_note_text,note_type,note_type_desc,label
0,78178720,2023-02-19 07:18:00,2023-02-19 05:52:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC2965753,11,2023-03-02 13:52:00+00:00,131349359373,857122433,2023-03-02 13:52:00+00:00,Provider Note Name: Wava Hussey...,ED Note,ED Provider Notes,1
1,78178720,2023-02-19 07:18:00,2023-02-19 05:52:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC2965753,11,2023-03-02 13:52:00+00:00,131349359373,857122432,2023-03-02 13:52:00+00:00,Provider Note Name: Wava Hussey...,ED Note,ED Provider Notes,1
2,13562038,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC737272,-29,2023-03-03 07:44:00+00:00,131351904872,866609862,2023-03-03 07:44:00+00:00,Provider Note Name: Bonnie Nord...,ED Note,ED Provider Notes,1
3,13562038,2023-04-01 00:56:00,2023-03-31 23:44:00,ED Provider,Intermediate Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC737272,-29,2023-03-03 07:44:00+00:00,131351904872,866609863,2023-03-03 07:44:00+00:00,Provider Note Name: Bonnie Nord...,ED Note,ED Provider Notes,1
4,18418582,2023-01-13 12:55:00,2023-01-13 11:06:00,ED Provider,Low Risk,BloodCultureAnalysis_Phase1_Tiebreaker.xlsx,JC1028634,-12,2023-01-01 19:06:00+00:00,131347122291,846841061,2023-01-01 19:06:00+00:00,Provider Note Name: Rhylee G Vo...,ED Note,ED Provider Notes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,23828635,2023-01-31 15:25:00,2023-01-31 16:32:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC1356726,6,2023-02-07 00:32:00+00:00,131348172318,852581578,2023-02-07 00:32:00+00:00,Provider Note Name: Moak: [000...,ED Note,ED Provider Notes,0
186,23828635,2023-01-31 15:25:00,2023-01-31 16:32:00,ED Provider,HighRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC1356726,6,2023-02-07 00:32:00+00:00,131348172318,852516321,2023-02-07 00:32:00+00:00,Provider Note Name: Moak: [000...,ED Note,ED Provider Notes,1
187,82062696,2023-02-26 20:58:00,2023-02-26 20:01:00,ED Provider,LowRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC3204260,-20,2023-02-07 04:01:00+00:00,131349746470,858655746,2023-02-07 04:01:00+00:00,Provider Note Name: Earl Zakhar...,ED Note,ED Provider Notes,1
188,82062696,2023-02-26 20:58:00,2023-02-26 20:01:00,ED Provider,LowRisk,BloodCultureAnalysis_Phase2_NickManoj.xlsx,JC3204260,-20,2023-02-07 04:01:00+00:00,131349746470,858654707,2023-02-07 04:01:00+00:00,Provider Note Name: Earl Zakhar...,ED Note,ED Provider Notes,1


In [112]:
human_merged_with_mrn_anon_id_mapping_with_culture_labels_cleaned = (
    human_merged_with_mrn_anon_id_mapping_with_culture_labels.copy()
)

# normalize spacing/case
col = human_merged_with_mrn_anon_id_mapping_with_culture_labels_cleaned["Risk of Bactermia"].str.strip().str.lower()

mapping = {
    "high risk": "High Risk",
    "highrisk": "High Risk",
    "intermediate risk": "Intermediate Risk",
    "intermediaterisk": "Intermediate Risk",
    "low risk": "Low Risk",
    "lowrisk": "Low Risk",
    "notdetermined": "Not Determined",
    "unclassified": "Not Determined",   # <-- collapse into same bucket
}

# apply mapping
human_merged_with_mrn_anon_id_mapping_with_culture_labels_cleaned["Risk of Bactermia"] = col.map(mapping)

# quick check
print(human_merged_with_mrn_anon_id_mapping_with_culture_labels_cleaned["Risk of Bactermia"].value_counts())


Risk of Bactermia
High Risk            91
Intermediate Risk    58
Low Risk             31
Not Determined       10
Name: count, dtype: int64


In [113]:
human_merged_with_mrn_anon_id_mapping_with_culture_labels_cleaned.groupby(["Risk of Bactermia", "label"]).size()

Risk of Bactermia  label
High Risk          0        13
                   1        78
Intermediate Risk  0         4
                   1        54
Low Risk           0        11
                   1        20
Not Determined     0         4
                   1         6
dtype: int64